## Market-timing Measures

Treynor-Mazuy:

$$ r_{i,t} - r_{f,t} = \alpha_i + \beta_i (r_{m,t} - r_{f,t}) + \gamma_i (r_{m,t} - r_{f,t})^2 + \varepsilon_{i,t} $$

Henriksson-Merton:

$$ r_{i,t} - r_{f,t} = \alpha_i + \beta_i (r_{m,t} - r_{f,t}) + \gamma_i (r_{m,t} - r_{f,t})^+ + \varepsilon_{i,t} $$

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go
from scipy.stats import norm

Let's simulate data with some level of market-timing ability.

In [2]:
# Parameters
ALPHA = 0
BETA  = 0.0
GAMMA = 1.0
NOBS  = 120

MN_MKTRF = 0.05/12
SD_MKTRF = 0.20/np.sqrt(12)
SD_EPS   = 0.05/np.sqrt(12)

In [3]:
# Simulate returns with market-timing ability
df = pd.DataFrame(dtype=float, columns = ['xret', 'mkt', 'eps'])
df['mkt']  = norm.rvs(loc=MN_MKTRF, scale=SD_MKTRF, size=NOBS)
df['eps']  = norm.rvs(loc=0, scale=SD_EPS, size=NOBS)
df['mkt_plus'] = np.where(df['mkt']>0, df['mkt'],0)
df['xret'] = ALPHA + BETA*df['mkt'] + GAMMA*df['mkt_plus'] + df['eps']

In [4]:
# Run market model
mm = sm.OLS(df['xret'], sm.add_constant(df['mkt'])).fit()
print(mm.summary())

                            OLS Regression Results                            
Dep. Variable:                   xret   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     230.2
Date:                Tue, 11 Apr 2023   Prob (F-statistic):           1.70e-29
Time:                        13:54:34   Log-Likelihood:                 287.52
No. Observations:                 120   AIC:                            -571.0
Df Residuals:                     118   BIC:                            -565.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0223      0.002     10.949      0.0

In [5]:
# Run Henriksson/Merton model
timing = sm.OLS(df['xret'], sm.add_constant(df[['mkt','mkt_plus']])).fit()
print(timing.summary())

                            OLS Regression Results                            
Dep. Variable:                   xret   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     314.3
Date:                Tue, 11 Apr 2023   Prob (F-statistic):           8.90e-48
Time:                        13:54:34   Log-Likelihood:                 333.72
No. Observations:                 120   AIC:                            -661.4
Df Residuals:                     117   BIC:                            -653.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.002      1.193      0.2

In [6]:
# Scatter plot with fitted market model and market-timing model
fig = go.Figure()
trace  = go.Scatter(x=df['mkt'], y=df['xret'], mode="markers", name = 'Returns')
fig.add_trace(trace)

# Market model
trace_mm= go.Scatter(x=df['mkt'], y=mm.fittedvalues, mode='lines',name='Market Model')
fig.add_trace(trace_mm)


# Market-timing model
sorted_mktvals = df['mkt'].sort_values()
sorted_timing = timing.fittedvalues.loc[sorted_mktvals.index]
trace_timing= go.Scatter(x=sorted_mktvals, y=sorted_timing, mode='lines',name='Market-Timing Model')
fig.add_trace(trace_timing)

# Formatting
fig.update_xaxes(title='Market Excess Return',tickformat=".2f")
fig.update_yaxes(title='Fund Excess Return',tickformat=".2f")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()